In [9]:
def timed(fn):
    from time import perf_counter
    
    def inner(*args, **kwargs):
        total_elapsed = 0
        for i in range(10): #hardcoded value of how many times we want to run the fn
            start = perf_counter()
            result = fn(*args, **kwargs)
            total_elapsed += (perf_counter() - start)
        avg_elapsed = total_elapsed / 10
        print(avg_elapsed)
        elapsed = end - start
        return result
    return inner

In [10]:
@timed
def my_func():
    ...

In [11]:
def timed1(fn, reps): #extra parameter
    from time import perf_counter
    
    def inner(*args, **kwargs):
        total_elapsed = 0
        for i in range(reps): #free variable
            start = perf_counter()
            result = fn(*args, **kwargs)
            total_elapsed += (perf_counter() - start)
        avg_elapsed = total_elapsed / reps
        print(avg_elapsed)
        elapsed = end - start
        return result
    return inner

In [12]:
my_func = timed1(my_func, 10) #OK

In [13]:
@timed1(10)
def my_func():
    ...
# WRONG

TypeError: timed1() missing 1 required positional argument: 'reps'

* **timed** is a function that returns that inner closure that contains our original function
* in order for this to work as intended:
```
@timed1(10)
def my_func():
    ...
```
* **timed(10)** will need to return our original timed decorator when called
```
dec = timed(10) # timed(10) returns a decorator
@dec
def my_fun():
    ...
```

In [14]:
def outer(reps):
    def timed2(fn):
        from time import perf_counter
        def inner(*args, **kwargs):
            total_elapsed = 0
            for i in range(reps): #free variable
                start = perf_counter()
                result = fn(*args, **kwargs)
                total_elapsed += (perf_counter() - start)
            avg_elapsed = total_elapsed / reps
            print(avg_elapsed)
            elapsed = end - start
            return result
        return inner
    return timed2

In [16]:
my_func = outer(10)(my_func) #OK

In [18]:
@outer(10)
def my_func():
    ...
    
# OK

# Decorator Factories
* The outer function is not itself a decorator, insteasd it returns a decorator when called
* And any arguments passed to outer can be referenced (as free variables) inside our decorator
* We call this outer function a **Decorator Factory** function
  * It is a function that creates a new decorator each time it is called

In [30]:
def timed2(reps):
    def dec(fn):
        from time import perf_counter
        def inner(*args, **kwargs):
            total_elapsed = 0
            for i in range(reps): #free variable
                start = perf_counter()
                result = fn(*args, **kwargs)
                total_elapsed += (perf_counter() - start)
            avg_elapsed = total_elapsed / reps
            print(round(avg_elapsed, 2), 's')
            print('reps: ', reps)
            return result
        return inner
    return dec

In [31]:
@timed2(10)
def my_func():
    ...
    
# OK

In [32]:
def calc_recursive_fib(n):
    if n <= 2:
        return 1
    else:
        return calc_recursive_fib(n-1) + calc_recursive_fib(n-2)

In [33]:
@timed2(15)
def fib(n):
    return calc_recursive_fib(n)

In [34]:
fib(28)

0.02 s
reps:  15


317811

# Decorator Application - Decorator Class

In [35]:
def my_dec(a, b):
    def dec(fn):
        def inner(*args, **kwargs):
            print("decorated function called a={0}, b={1}".format(a,b))
            return fn(*args, **kwargs)
        return inner
    return dec

In [36]:
@my_dec(10, 20)
def my_func(s):
    print("Hello {0}".format(s))

In [37]:
my_func('World')

decorated function called a=10, b=20
Hello World


In [39]:
class MyClass:
    def __init__(self, a, b):
        self.a = a
        self.b = b
        
    def __call__(self, c):
        print("called a={0}, b={1}, c={2}".format(self.a, self.b, c))

In [40]:
obj = MyClass(10, 20)

In [41]:
obj

In [42]:

obj.__call__(100)

called a=10, b=20, c=100


In [43]:
obj(100)

called a=10, b=20, c=100


In [47]:
class MyClass:
    def __init__(self, a, b):
        self.a = a
        self.b = b
        
    def __call__(self, fn):
        def inner(*args, **kwargs):
            print("decorated function called a={0}, b={1}".format(self.a, self.b))
            return fn(*args, **kwargs)
        return inner

In [48]:
@MyClass(10, 20)
def my_func(s):
    print('Hello {0}'.format(s))

In [49]:
my_func('World')

decorated function called a=10, b=20
Hello World


In [50]:
obj = MyClass(10, 20)
def my_func(s):
    print('Hello {0}'.format(s))

In [51]:
my_func = obj(my_func)

In [52]:
my_func(10)

decorated function called a=10, b=20
Hello 10


In [53]:
from fractions import Fraction

In [56]:
f = Fraction(2,3)

In [57]:
f.denominator

3

In [58]:
f.numerator

2

In [59]:
Fraction.speak = 100

In [60]:
f.speak

100